# Oil Production Toy Problems

These problems illustrate the use of oil production cost data in the openLCA database and the construction of more complex optimisation problems using openLCA data.

# Problem Statement

An oil company wishes to find the cheapest region to extract 10000 tonnes of crude oil from onshore oil plants in the Middle East, Russia or Africa and then transport the oil to a refinery by sea tanker. The distance from the extraction site to the refinery in each region is given in the table below.

Region | Distance / km
-- | --
Middle East | 200
Russia | 600
Africa | 400

## Assumptions

* We ignore the environmental impact of extraction, transport and building the refinery.
* We ignore the cost of building the refinery initially.

# Analysis

First we find the three crude oil production processes in the openLCA database.

In [1]:
import mola.dataimport as di
import mola.dataview as dv
conn = di.get_sqlite_connection()

In [2]:
oil_production_dfr = dv.get_processes(conn, name=['petroleum production, onshore%'])
oil_production_dfr

SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE 'petroleum production, onshore%'


,ID,REF_ID,NAME,PROCESS_TYPE,LOCATION,F_QUANTITATIVE_REFERENCE
0,7076520,81d22f90-89fc-3f04-845e-97615867d8f9,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Russian Federation,7076522
1,10012499,2c08823a-4575-30ba-bf61-36e2fd4d56e6,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Rest-of-World,10012501
2,15742513,c1827905-bfad-3fa1-b00b-1303f970ae92,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Middle East,15742515
3,29432025,cd177b7d-e908-3e69-b40c-4827b4abaa4d,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Africa,29432027


We can explicitly find the cost of producing oil at the three locations using the following `mola` function.

In [3]:
production_ref_id = [
    '81d22f90-89fc-3f04-845e-97615867d8f9',
    'c1827905-bfad-3fa1-b00b-1303f970ae92',
    'cd177b7d-e908-3e69-b40c-4827b4abaa4d'
]
dv.get_process_product_flow_costs(conn, process_ref_ids=production_ref_id)

SELECT "TBL_PROCESSES"."REF_ID" "PROCESS_REF_ID","TBL_PROCESSES"."NAME" "PROCESS_NAME","TBL_LOCATIONS"."NAME" "LOCATION","TBL_FLOWS"."REF_ID" "FLOW_REF_ID","TBL_FLOWS"."NAME" "FLOW_NAME","sq0"."COST_VALUE","TBL_CURRENCIES"."NAME" "CURRENCY","TBL_UNITS"."NAME" "UNITS" FROM (SELECT "F_OWNER","COST_VALUE","F_CURRENCY","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (SELECT "ID" FROM "TBL_PROCESSES" WHERE "REF_ID" IN ('81d22f90-89fc-3f04-845e-97615867d8f9','c1827905-bfad-3fa1-b00b-1303f970ae92','cd177b7d-e908-3e69-b40c-4827b4abaa4d'))) "sq0" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="sq0"."F_FLOW" LEFT JOIN "TBL_PROCESSES" ON "TBL_PROCESSES"."ID"="sq0"."F_OWNER" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" LEFT JOIN "TBL_UNITS" ON "TBL_UNITS"."ID"="sq0"."F_UNIT" LEFT JOIN "TBL_CURRENCIES" ON "TBL_CURRENCIES"."ID"="sq0"."F_CURRENCY" WHERE "TBL_FLOWS"."FlOW_TYPE"='PRODUCT_FLOW'


,PROCESS_REF_ID,PROCESS_NAME,LOCATION,FLOW_REF_ID,FLOW_NAME,COST_VALUE,CURRENCY,UNITS
0,81d22f90-89fc-3f04-845e-97615867d8f9,"petroleum production, onshore | petroleum | AP...",Russian Federation,e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,petroleum,0.192,Euro,kg
1,c1827905-bfad-3fa1-b00b-1303f970ae92,"petroleum production, onshore | petroleum | AP...",Middle East,e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,petroleum,0.192,Euro,kg
2,cd177b7d-e908-3e69-b40c-4827b4abaa4d,"petroleum production, onshore | petroleum | AP...",Africa,e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,petroleum,0.192,Euro,kg


OpenLCA reports the cost is the same in each country so it would appear to be an assumption. The same pattern is seen for costs for other processes in openLCA when one looks across location.

In openLCA there is only only one global transport process for petroleum by sea tanker. We obtain the transport process from the database below.

In [4]:
oil_transport_dfr = dv.get_processes(conn, name=['transport, freight, sea, tanker for petroleum %'])
oil_transport_dfr

SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE 'transport, freight, sea, tanker for petroleum %'


,ID,REF_ID,NAME,PROCESS_TYPE,LOCATION,F_QUANTITATIVE_REFERENCE
0,33780409,8d82d02e-f52f-41f9-aa10-ae607746b04e,"transport, freight, sea, tanker for petroleum ...",LCI_RESULT,Global,33780411


In [5]:
transport_ref_id = [
    '8d82d02e-f52f-41f9-aa10-ae607746b04e'
]
dv.get_process_product_flow_costs(conn, process_ref_ids=transport_ref_id)

SELECT "TBL_PROCESSES"."REF_ID" "PROCESS_REF_ID","TBL_PROCESSES"."NAME" "PROCESS_NAME","TBL_LOCATIONS"."NAME" "LOCATION","TBL_FLOWS"."REF_ID" "FLOW_REF_ID","TBL_FLOWS"."NAME" "FLOW_NAME","sq0"."COST_VALUE","TBL_CURRENCIES"."NAME" "CURRENCY","TBL_UNITS"."NAME" "UNITS" FROM (SELECT "F_OWNER","COST_VALUE","F_CURRENCY","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (SELECT "ID" FROM "TBL_PROCESSES" WHERE "REF_ID" IN ('8d82d02e-f52f-41f9-aa10-ae607746b04e'))) "sq0" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="sq0"."F_FLOW" LEFT JOIN "TBL_PROCESSES" ON "TBL_PROCESSES"."ID"="sq0"."F_OWNER" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" LEFT JOIN "TBL_UNITS" ON "TBL_UNITS"."ID"="sq0"."F_UNIT" LEFT JOIN "TBL_CURRENCIES" ON "TBL_CURRENCIES"."ID"="sq0"."F_CURRENCY" WHERE "TBL_FLOWS"."FlOW_TYPE"='PRODUCT_FLOW'


,PROCESS_REF_ID,PROCESS_NAME,LOCATION,FLOW_REF_ID,FLOW_NAME,COST_VALUE,CURRENCY,UNITS
0,8d82d02e-f52f-41f9-aa10-ae607746b04e,"transport, freight, sea, tanker for petroleum ...",Global,774bc814-70cf-4389-8bf8-e6435174c72e,"transport, freight, sea, tanker for petroleum",0.000314,Euro,t*km


The optimal region is the Middle East, since the cost of production is the same according to openLCA in each region and the Middle East has the shortest distance to the refinery.

## Sets and Parameters

In this section we build the sets and parameters for `mola`. First we load the specification.

In [6]:
import mola.specification5 as ms
from importlib import reload
spec = ms.ScheduleSpecification()

Then we set up a configuration file for the sets.

In [7]:
import mola.input as mi
cost_set_file_name = 'Configuration/cost_set_data.json'
user_sets = mi.get_model_user_sets(spec, cost_set_file_name)
user_sets

Model sets saved


{'F_m': ['e6aad2de-0b1b-49c3-a0c4-797ba34d87e5'],
 'F_s': [],
 'F_t': ['774bc814-70cf-4389-8bf8-e6435174c72e'],
 'D': ['d1'],
 'T': ['t1'],
 'K': ['t1'],
 'P_m': ['81d22f90-89fc-3f04-845e-97615867d8f9',
  'c1827905-bfad-3fa1-b00b-1303f970ae92',
  'cd177b7d-e908-3e69-b40c-4827b4abaa4d'],
 'P_t': ['8d82d02e-f52f-41f9-aa10-ae607746b04e'],
 'P_s': [],
 'KPI': [],
 'OBJ': ['environment', 'cost'],
 'F': [],
 'P': []}

In [8]:
import mola.dataview as dv
import mola.dataimport as di
import pandas as pd
from pyomo.environ import *
conn = di.get_sqlite_connection()
lookup = dv.LookupTables(conn)

We show a widget that loads and saves the user configuration of sets in a tabbed interface. Existing user configuration data is loaded if there is saved model data file.

In [9]:
import mola.widgets as mw
lookups = {'F_m': lookup.get('flows'), 'P_m': lookup.get('processes'), 'F_t': lookup.get('flows'), 
           'P_t': lookup.get('processes'), 'F_s': lookup.get('flows'), 
           'P_s': lookup.get('processes'), 'KPI': lookup.get('KPI')}
vbox, tab = mw.get_sets(spec, lookups, cost_set_file_name)
vbox

Below, we load the sets defined from the JSON file and incorporate any non-user defined sets.

In [10]:
set_data = mi.get_model_user_sets(spec, cost_set_file_name)
set_data

Model sets saved


{'F_m': ['e6aad2de-0b1b-49c3-a0c4-797ba34d87e5'],
 'F_s': [],
 'F_t': ['774bc814-70cf-4389-8bf8-e6435174c72e'],
 'D': ['d1'],
 'T': ['t1'],
 'K': ['t1'],
 'P_m': ['81d22f90-89fc-3f04-845e-97615867d8f9',
  'c1827905-bfad-3fa1-b00b-1303f970ae92',
  'cd177b7d-e908-3e69-b40c-4827b4abaa4d'],
 'P_t': ['8d82d02e-f52f-41f9-aa10-ae607746b04e'],
 'P_s': [],
 'KPI': [],
 'OBJ': ['environment', 'cost'],
 'F': [],
 'P': []}

The parameters are defined using the widget below.

In [11]:
import qgrid
cost_parameters_file_name = 'Configuration/cost_parameters_data.json'
param_dfr, param_dict = mi.get_model_user_parameters(spec, set_data, cost_parameters_file_name)
mw.get_parameters(param_dfr, cost_parameters_file_name)

## Model build

To build the model we need to retrieve the cost of each flow from the database. This is done using the `populate` method
of the `Specification` object.

In [12]:
json_files = [cost_set_file_name, cost_parameters_file_name]
model_instance = spec.populate(json_files)

ERROR: Constructing component 'UM' from
    data={'81d22f90-89fc-3f04-845e-97615867d8f9': 'kg',
    'c1827905-bfad-3fa1-b00b-1303f970ae92': 'kg',
    'cd177b7d-e908-3e69-b40c-4827b4abaa4d': 'kg'} failed:
        RuntimeError: Failed to set value for param=UM,
        index=81d22f90-89fc-3f04-845e-97615867d8f9, value=kg.
    	source error message="Index '81d22f90-89fc-3f04-845e-97615867d8f9' is not
    	valid for indexed component 'UM'"


RuntimeError: Failed to set value for param=UM, index=81d22f90-89fc-3f04-845e-97615867d8f9, value=kg.
	source error message="Index '81d22f90-89fc-3f04-845e-97615867d8f9' is not valid for indexed component 'UM'"

We can see that the cost data for the product flows has been loaded from the openLCA database.

In [ ]:
model_instance.phi.pprint()

# Solution

We deactivate environment and combined objectives.

In [ ]:
model_instance.obj1.deactivate()
model_instance.obj2.activate()
model_instance.obj.deactivate()

In [ ]:
opt = SolverFactory("glpk")
results = opt.solve(model_instance)
results.write()

In [ ]:
import mola.output as mo
pd.set_option('display.max_colwidth', 1000)
mo.get_entity(model_instance.Flow, lookup, drop=True, units=['P_m'])

In [ ]:
mo.get_entity(model_instance.Specific_Material_Transport_Flow, lookup, drop=True, units=['P_m'])

In [ ]:
mo.get_entity(model_instance.Specific_Transport_Flow, lookup, drop=True, units=['P_t'])